Pure Python code

In [3]:
#!/usr/bin/env python3
import sys, timeit
import matplotlib.pyplot as plt
import scipy.sparse
import scipy.sparse.linalg
import numpy as np

def solver_FE(I, a, Nt, Nx, L, dt, F, T, user_action=None):

    Nt = int(round(T/float(dt)))
    t = np.linspace(0, Nt*dt, Nt+1)   # Mesh points in time
    dx = np.sqrt(a*dt/F)
    Nx = int(round(L/dx))
    x = np.linspace(0, L, Nx+1)       # Mesh points in space
    # Make sure dx and dt are compatible with x and t
    dx = x[1] - x[0]
    dt = t[1] - t[0]

    u   = np.zeros(Nx+1)   # solution array
    u_n = np.zeros(Nx+1)   # solution at t-dt

    # Set initial condition
    for i in range(0,Nx+1):
        u_n[i] = I(x[i],L)

    if user_action is not None:
        user_action(u_n, x, t, 0)

    for n in range(0, Nt):
        # Update all inner points
        for i in range(1, Nx):
                u[1:Nx] = u_n[1:Nx] + F*(u_n[0:Nx-1] - 2*u_n[1:Nx] + u_n[2:Nx+1])         # Insert boundary conditions
        u[0] = 0;  u[Nx] = 0
        if user_action is not None:
            user_action(u, x, t, n+1)

        # Switch variables before next step
        u_n, u = u, u_n

def plottingFE(I, a, Nt, Nx, L, dt, F, T, animate=True):
    def plot_u(u, x, t, n):
        plt.plot((x)**2, 2/3*u/(x+1E-10), 'r-')
        plt.xlabel(r'$R/R_0$')
        plt.ylabel(r'$\Sigma$')
        plt.xlim(0,2)
        #plt.show()
    user_action = plot_u if animate else lambda u,x,t,n: None

    return solver_FE(I, a, Nt, Nx, L, dt, F, T, user_action=plot_u)
def I(x,L):
    """Surface density profile as initial condition."""
    return np.sqrt(2.3*x)*(L/2)**(1/4)*np.exp(-3.75*((-(x)**2+L/2.0)**2)/0.05**2)

def SurfaceDensityProfile( F, Nx):
    L = 2. # size of the x axis
    a = 12 # diffusion coefficient
    T = 1 # time
    Sigma=1
    # Compute dt from Nx and F
    dx = L/Nx;  dt = F/a*dx**2
    Nt = int(round(T/float(dt)))
    
    return plottingFE(I, a, Nt, Nx, L, dt, F, T, animate=True)

%timeit(SurfaceDensityProfile(0.5,50))

UsageError: Line magic function `%timeit(SurfaceDensityProfile(0.5,50))` not found.


Numba-CUDA.jit

In [2]:
#Required NVIDIA libraries
!find / -iname 'libdevice'
!find / -iname 'libnvvm.so'

find: ‘/etc/cups/ssl’: Permission denied
find: ‘/etc/ssl/private’: Permission denied
find: ‘/etc/polkit-1/localauthority’: Permission denied
/usr/lib/cuda/nvvm/libdevice
/usr/lib/nvidia-cuda-toolkit/libdevice
find: ‘/proc/tty/driver’: Permission denied
find: ‘/proc/1/task/1/fd’: Permission denied
find: ‘/proc/1/task/1/fdinfo’: Permission denied
find: ‘/proc/1/task/1/ns’: Permission denied
find: ‘/proc/1/fd’: Permission denied
find: ‘/proc/1/map_files’: Permission denied
find: ‘/proc/1/fdinfo’: Permission denied
find: ‘/proc/1/ns’: Permission denied
find: ‘/proc/2/task/2/fd’: Permission denied
find: ‘/proc/2/task/2/fdinfo’: Permission denied
find: ‘/proc/2/task/2/ns’: Permission denied
find: ‘/proc/2/fd’: Permission denied
find: ‘/proc/2/map_files’: Permission denied
find: ‘/proc/2/fdinfo’: Permission denied
find: ‘/proc/2/ns’: Permission denied
find: ‘/proc/3/task/3/fd’: Permission denied
find: ‘/proc/3/task/3/fdinfo’: Permission denied
find: ‘/proc/3/task/3/ns’: Permission denied
find

In [ ]:
#!/usr/bin/env python3
import timeit, time
import matplotlib.pyplot as plt
import numpy as np
from numba import jit, prange
from numba.types import float32, int32, string
from numba import cuda
#jit([float32(float32, int32, float32, float32, int32, float32, float32, int32, string)])
@cuda.jit(device=True)
def solver_FE_jit(I, a,t, Nt, Nx, L, dt, F, T, user_action=None):

    Nt = int(round(T/float(dt)))
    t = np.linspace(0, Nt*dt, Nt+1)   # Mesh points in time
    dx = np.sqrt(a*dt/F)
    Nx = int(round(L/dx))
    x = np.linspace(0, L, Nx+1)       # Mesh points in space
    # Make sure dx and dt are compatible with x and t
    dx = x[1] - x[0]
    dt = t[1] - t[0]

    u   = np.zeros(Nx+1)   # solution array
    u_n = np.zeros(Nx+1)   # solution at t-dt

    # Set initial condition
    for i in prange(0,Nx+1):
        u_n[i] = I(x[i])

    if user_action is not None:
        user_action(u_n, x, t, 0)

    for n in prange(0, Nt):
        # Update all inner points
        for i in prange(1, Nx):
                u[1:Nx] = u_n[1:Nx] + F*(u_n[0:Nx-1] - 2*u_n[1:Nx] + u_n[2:Nx+1])         # Insert boundary conditions
        u[0] = 0;  u[Nx] = 0
        if user_action is not None:
            user_action(u, x, t, n+1)

        # Switch variables before next step
        u_n, u = u, u_n

def plottingFE_jit(I, a, Nt, Nx, L, dt, F, T, animate=True):
    def plot_u(u, x, t, n):
        plt.plot((x)**2, 2/3*u/(x+1E-10), 'r-')
        plt.xlabel(r'$R/R_0$')
        plt.ylabel(r'$\Sigma (10^6 g/cm^2)$')
        plt.xlim(0,2)
        #plt.show()
    user_action = plot_u if animate else lambda u,x,t,n: None

    return solver_FE_jit(I, a, Nt, Nx, L, dt, F, T, user_action=plot_u)

def SurfaceDensityProfile_jit( F, Nx):
    L = 2. # Size of x-axis
    a = 12 # diffusion coefficient
    T = 1 #time
    sigma=0.05
    # Compute dt from Nx and F
    dx = L/Nx;  dt = F/a*dx**2
    Nt = int(round(T/float(dt)))
    def I(x):
        """Surface density profile as initial condition."""
        return np.sqrt(2.3*x)*(L/2)**(1/4)*np.exp(-3.75*((-(x)**2+L/2.0)**2)/sigma**2)

    return plottingFE_jit(I, a, Nt, Nx, L, dt, F, T, animate=True)
print("--------------python-------------------")
%timeit(cuda.jit(device=True)(SurfaceDensityProfile_jit))
print("--------------nopython-------------------")
%timeit(cuda.jit(nopython=True)(SurfaceDensityProfile_jit))

--------------python-------------------
10000 loops, best of 5: 40.7 µs per loop
--------------nopython-------------------
The slowest run took 5.94 times longer than the fastest. This could mean that an intermediate result is being cached.
100000 loops, best of 5: 8.91 µs per loop


Using Multiprocessing library in Python


In [ ]:
if __name__ == "__main__":
    import multiprocessing
    # creating processes
    p1 = multiprocessing.Process(target=SurfaceDensityProfile, args=(0.5,50))
    # starting process 1
    p1.start()  
    # wait until process 1 is finished
    p1.join()
    print('Multiprocessing time:') 
    %timeit(p1)

Multiprocessing time:
10000000 loops, best of 5: 30.2 ns per loop


MPI4PY

In [ ]:
!pip3 install mpi4py

In [ ]:
#!/usr/bin/env python3
import timeit, time
import matplotlib.pyplot as plt
import numpy as np
from mpi4py import MPI

def parallel():
  comm = MPI.COMM_WORLD
  rank = comm.Get_rank()
  nprocs = comm.Get_size()
  totalStartTime = time.time()
  for i in range(rank, 50, nprocs):
    sdp=SurfaceDensityProfile(0.5,50)
  print("Total time %f seconds" %(time.time() - totalStartTime))

if __name__=='__main__':
  parallel()
MPI.Finalize()